# Libraries

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [7]:
%run Data_preprocessing_v1.ipynb

************************************************************
There are 6 set of X
X_train_selected_t1, X_train_selected_t2, X_valid_selected_t1,X_valid_selected_t2,X_test_selected_t1,X_test_selected_t2
------------------------------------------------------------
Normalized version
X_train_selected_t1_norm, X_train_selected_t2_norm, X_valid_selected_t1_norm,X_valid_selected_t2_norm,X_test_selected_t1_norm,X_test_selected_t2_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [8]:
print(X_train_selected_t2_norm.shape)
print(X_valid_selected_t2_norm.shape)
print(X_test_selected_t2_norm.shape)

(16760, 61)
(2394, 61)
(4790, 61)


# Hyperparameters

In [50]:
input_size = 61
sequence_length = 1
output_size = 1 # live or dead
num_layers = 3
hidden_size=30
learning_rate = 0.001
batch_size = 64
num_epochs = 500
l2_lamb = 0.0005

# Create JusNN

In [14]:
class JusNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(JusNN, self).__init__()
        
        self.convlayer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=2, stride=1, padding=0, dilation=0),
            nn.AvgPool1d(kernel_size=2,stride=1)
            nn.Conv1d(in_channels=6, out_channels=3, kernel_size=1, stride=1, padding=0, dilation=0),
            nn.ReLU(),
        )
        
        self.fclayers = nn.Sequential(
            nn.Linear(input_size, 60),
            nn.ReLU()
            nn.Linear(60, 31),
        )
        
        self.batchnorm1 = nn.BatchNorm1d(61)
        #self.batchnorm2 = nn.BatchNorm1d(30)
        #self.dropout = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(31, 1)
        
    def forward(self, x):
         x1 = self.batchnorm1(x)
         x1 = F.leaky_relu(self.fc1(x1))
            
         x2 = self.batchnorm1(x)
         x2 = self.convlayer(x2)
            
         x3 = x1+x2
         x3 = self.fc1(x3)
         return x3

SyntaxError: invalid syntax (2058534331.py, line 8)

# Data transform

In [52]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [53]:
# X_shape(X) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def X_shape(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    X_tensor = torch.split(X_tensor,1)
    X_tensor = torch.stack(X_tensor)
    print("X now in shape of",X_tensor.shape)
    return X_tensor

# y_tensor(y) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32).reshape(len(y),1)
    print("y now in shape of",y.shape)
    return y

In [54]:
# X should be in smaples, 1, squence, rows
X1 = X_shape(X_train_selected_t2_norm)
y1 = y_tensor(y_train_t2.iloc[:,-1])

train_datasets = CusDatasetLoader(X1, y1)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

x,y = train_datasets[0]
print(x.shape)
print(y.shape)

X now in shape of torch.Size([16760, 1, 61])
y now in shape of torch.Size([16760, 1])
torch.Size([1, 61])
torch.Size([1])


# Model train

In [55]:
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)

In [56]:
criterion = nn.MSELoss()
optimizer = optim.Adadelta(model.parameters(), lr = learning_rate)

In [57]:
losses=[]
acc_list=[]
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.permute(0,1,2)
        #print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        targets = torch.squeeze(targets, 1)
        #print(targets)
        
        # forward
        scores = model(data)
        #print(scores.shape)
        loss = criterion(scores, targets)
        l2_lambda = l2_lamb
        l2_norm = sum(p.pow(2.0).sum()
                  for p in model.parameters())
 
        loss = loss + l2_lambda * l2_norm
    
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    if epoch%5 == 0:
        losses.append(loss.detach().numpy() )
        print(f'Epoch {epoch:03}: | Loss: {loss:.5f} ')
        

Epoch 000: | Loss: 14.93716 
Epoch 005: | Loss: 13.15120 
Epoch 010: | Loss: 16.31759 
Epoch 015: | Loss: 11.50748 
Epoch 020: | Loss: 6.69250 
Epoch 025: | Loss: 7.00234 
Epoch 030: | Loss: 5.25601 
Epoch 035: | Loss: 3.07440 
Epoch 040: | Loss: 3.51244 
Epoch 045: | Loss: 4.72005 
Epoch 050: | Loss: 4.92757 
Epoch 055: | Loss: 3.90265 
Epoch 060: | Loss: 5.93129 
Epoch 065: | Loss: 5.17600 
Epoch 070: | Loss: 5.48070 
Epoch 075: | Loss: 5.79294 
Epoch 080: | Loss: 5.50427 
Epoch 085: | Loss: 3.88706 
Epoch 090: | Loss: 4.83165 
Epoch 095: | Loss: 4.10547 
Epoch 100: | Loss: 3.13122 
Epoch 105: | Loss: 4.86404 
Epoch 110: | Loss: 3.58423 
Epoch 115: | Loss: 5.38918 
Epoch 120: | Loss: 4.28800 
Epoch 125: | Loss: 4.01491 
Epoch 130: | Loss: 4.21155 
Epoch 135: | Loss: 4.86178 
Epoch 140: | Loss: 4.26680 
Epoch 145: | Loss: 3.94521 
Epoch 150: | Loss: 3.73941 
Epoch 155: | Loss: 4.63653 
Epoch 160: | Loss: 3.71185 
Epoch 165: | Loss: 3.61252 
Epoch 170: | Loss: 3.81926 
Epoch 175: | Los

# Model RMSE

In [58]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def check_rmse(loader, model):

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)

            scores = model(x)
            res = rmse(scores,targets)
            print(res)
            
    # Toggle model back to train
    model.train()
    
    

In [59]:
X2 = X_shape(X_valid)
y2 = y_tensor(y_valid_t1.iloc[:,-1])

Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
print(f"RMSE on test set: {check_rmse(Valid_loader, model)*100:.2f} %")

NameError: name 'X_valid' is not defined

# Test

In [ ]:
def test(loader, model):
    # Set model to eval
    model.eval()
    res = torch.tensor([], dtype=torch.int64)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)
            print(y.shape)

            scores = model(x)
            _, predictions = scores.max(1)
            res =  torch.cat((predictions, res), 0)  
    model.train()
   
    return res

In [ ]:
X_test

In [ ]:
X_test_final = X_shape(X_test)
y_final = torch.zeros([4790,1])

test_datasets = CusDatasetLoader(X_test_final, y_final)
test_loader = DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=True)

result = test(test_loader, model)
result

In [ ]:

#df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
res1 = pd.DataFrame(result, columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')


#

#

#